In [65]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

__all__ = [
    'load_heartrate'
]


def creat_heartrate(dataset,window, as_series=False):
    """ Heart Rate Data from MIT dataset:
    
    Heart rhythms in health and disease display complex dynamics. 
    The clinical data suggest that concepts developed in nonlinear mathematics, such as bifurcations and chaos,will be appropriate to describe some of these complex phenomena. 
    Careful analysis will be needed to establish the presence of deterministic chaos in cardiac rhythms. 
    Data that appear highly periodic such as normal sinus rhythm may in reality be quite variable. 
    In contrast, chaotic-appearing rhythms such as ventricular fibrillation often contain strong periodicities.
    
    References
    ----------
    
    Goldberger AL, Rigney DR. Nonlinear dynamics at the bedside. In: Glass L, Hunter P, McCulloch A, eds. Theory of Heart: Biomechanics, Biophysics, and Nonlinear Dynamics of Cardiac Function. New York: Springer-Verlag, 1991, pp. 583-605.
    
    --------
    
    A sample of heartrate data borrowed from an `MIT database <http://ecg.mit.edu/time-series/>`_. 
    The sample consists of 150 evenly spaced (0.5 seconds) heartrate measurements.
    There are two groups(health and disease ) with measurements of heart beat rate, i.e., 6*300 measurements for each group.
    
    Parameters
    ----------
    as_series : bool, optional (default=False)
        Whether to return a Pandas series. If False, will return a 1d
        numpy array.

    Original_data:
    
    Returns
    -------
    dataset : array-like, shape=(n_samples,)
        The heartrate vector.
        
    Examples
    --------
    >>> from datetime import datetime, timedelta
    >>> import pandas as pd
    >>> import numpy as np
    >>> dataset = pd.read_csv('HeartRate_MIT.csv',sep=',',header='infer')
    >>> creat_heartrate(dataset)
    --------
    
    """
    
    # Define the start and end time
    start_time = datetime(2023, 10, 14, 0, 0, 0)  # Start at midnight
    T=600
    end_time = datetime(2023, 10, 14, 0, 8, 15)  # End at 11:59:59 PM
    
    # Define the time step (1 second in this case)
    time_step = timedelta(seconds=5)
    
    # Initialize an empty list to store the time series
    time_series = []
    
    # Generate the time series
    current_time = start_time
    n=0
    while n< T:
        time_series.append(current_time.strftime("%H:%M:%S"))  # Format and add to the list
        current_time += time_step
        n += 1
    
    '''
    # Print the time series
    for timestamp in time_series:
        print(timestamp)
    '''
    Dataa=pd.DataFrame(time_series,columns=['Time'])
    DataT=pd.DataFrame(time_series,columns=['Time'])
    for col in range(len(dataset.columns)):
        data= dataset[dataset.columns[col]] 
        data= np.array(data).reshape(3,600)
        # print(data)
        # np.asarray(ds1.outputs).reshape(-1).tolist()
        for n in range(3):
            if col ==0:
                colname = ['Health_'+str(n+1)]
            else:
                colname = ['Disease_'+str(n+1)]
            Data=pd.DataFrame(np.array(pd.DataFrame(data).iloc[n,:]),columns=colname)
            Dataa=pd.concat([Dataa, Data], axis=1)
        ave=np.average(np.transpose(Dataa.drop('Time',axis=1)), axis=0) 
        print(ave)
        ave = pd.DataFrame(ave,columns=['AVE_'+dataset.columns[col]])
        Dataa=pd.concat([Dataa,ave], axis=1)
        Dataaa=pd.concat([DataT, Dataa], axis=1)
    if as_series:
        return pd.Series(data)
    data = pd.DataFrame(data = Dataaa.T.drop_duplicates().T )
    data.to_csv('HeartRate_MIT_Ave.csv',index=False) 
    Rawdata_Health = pd.read_csv('HeartRate_MIT_Ave.csv', index_col=['Time'], parse_dates=['Time'],usecols=['Time','AVE_Health'])
    Rawdata_Disease = pd.read_csv('HeartRate_MIT_Ave.csv', index_col=['Time'], parse_dates=['Time'],usecols=['Time','AVE_Disease'])
    # trail-rolling average transform
    rolling_Health = Rawdata_Health.rolling(window=window)
    rolling_mean_H = rolling_Health.mean()
    rolling_Disease = Rawdata_Disease.rolling(window=window)
    rolling_mean_D = rolling_Disease.mean()
    print([rolling_mean_H])
    Pred_H = pd.DataFrame(np.array(rolling_mean_H),columns=['Prediction_Health'])
    Pred_D = pd.DataFrame(np.array(rolling_mean_D),columns=['Prediction_Disease'])
    DataP=pd.concat([data, Pred_H], axis=1)
    DataP=pd.concat([DataP, Pred_D], axis=1)
    DataP=DataP.drop('AVE_Health',axis=1).drop('AVE_Disease',axis=1)
    DataP.to_csv('HeartRate_MIT_Pred.csv',index=False) 
    #print(DataP)
    
    return DataP

In [66]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
dataset = pd.read_csv('HeartRate_MIT.csv',sep=',',header='infer')
window = 2
creat_heartrate( dataset,window)

[ 87.71753333  87.3951      87.4222      87.86283333  87.90466667
  87.4288      87.27556667  87.23803333  87.2967      87.63463333
  87.71333333  87.08636667  86.88636667  87.35003333  88.018
  88.46693333  88.6089      89.65903333  90.81706667  91.01003333
  90.57393333  89.8614      89.78996667  90.5425      91.0962
  90.87166667  90.3669      90.34023333  90.5981      91.63053333
  92.1237      91.4112      90.8826      90.78493333  90.6197
  90.1565      90.08726667  90.49903333  90.7274      90.3233
  90.54963333  91.02153333  91.0139      91.9244      93.00403333
  93.18493333  92.5061      91.4577      91.0678      91.6452
  92.86746667  93.35663333  93.2628      93.2417      93.1516
  93.3152      93.25836667  92.4828      92.07046667  92.44506667
  92.99123333  93.50003333  93.54336667  93.40676667  93.1205
  93.4656      94.22556667  94.32196667  94.15826667  94.57786667
  94.88923333  94.60873333  94.67443333  94.42123333  93.81273333
  93.78973333  93.719       93.0939    

/var/folders/3m/79tl3y611yb3nh_j7ws55tvw0000gn/T/ipykernel_78595/3506787667.py:102: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Rawdata_Health = pd.read_csv('HeartRate_MIT_Ave.csv', index_col=['Time'], parse_dates=['Time'],usecols=['Time','AVE_Health'])
/var/folders/3m/79tl3y611yb3nh_j7ws55tvw0000gn/T/ipykernel_78595/3506787667.py:103: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Rawdata_Disease = pd.read_csv('HeartRate_MIT_Ave.csv', index_col=['Time'], parse_dates=['Time'],usecols=['Time','AVE_Disease'])


,Time,Health_1,Health_2,Health_3,Disease_1,Disease_2,Disease_3,Prediction_Health,Prediction_Disease
0,00:00:00,84.2697,86.0997,92.7832,91.4634,101.957,88.652,NaN,NaN
1,00:00:05,84.2697,85.1549,92.7607,91.4634,102.651,89.9465,87.556317,90.470274
2,00:00:10,84.0619,84.7458,93.4589,91.1834,102.552,89.0909,87.408650,90.439743
3,00:00:15,85.6542,84.601,93.3333,91.8788,102.329,90.0317,87.642517,90.586138
4,00:00:20,87.2093,84.4511,92.0536,91.1772,102.516,90.343,87.883750,90.810407
...,...,...,...,...,...,...,...,...,...
595,00:49:35,88.5214,92.4062,103.79,101.961,86.3133,98.6842,94.926367,95.143088
596,00:49:40,88.2604,94.2328,101.623,101.897,86.7052,98.6842,94.805633,95.192140
597,00:49:45,88.2833,95.2028,99.5679,101.633,86.293,99.0005,94.528367,95.031417
598,00:49:50,88.5444,94.9367,99.1835,101.757,85.9039,99.3273,94.286433,94.871869
